In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [8]:
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [7]:
#quick overview
datasets = {"Customers": customers, "Products": products, "Transactions": transactions}
for name, df in datasets.items():
    print(f"\n--- {name} Dataset ---")
    print(df.info())
    print(df.describe())
    print(df.head(3))


--- Customers Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None
       CustomerID      CustomerName         Region  SignupDate
count         200               200            200         200
unique        200               200              4         179
top         C0001  Lawrence Carroll  South America  2024-11-11
freq            1                 1             59           3
  CustomerID      CustomerName         Region  SignupDate
0      C0001  Lawrence Carroll  South America  2022-07-10
1      C0002    Elizabeth Lutz           Asia  2022-02-13
2      C0003    Michael Rivera  South America  2024-03-07

--- Products Dataset ---

In [6]:
#checking for missing values
print("\n--- Missing Values Summary ---")
for name, df in datasets.items():
    print(f"{name}:")
    print(df.isnull().sum())


--- Missing Values Summary ---
Customers:
CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
Products:
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
Transactions:
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [9]:
# Merging datasets for comprehensive analysis
merged_data = transactions.merge(products, on="ProductID", how="inner")
merged_data = merged_data.merge(customers, on="CustomerID", how="inner")
print("\n--- Merged Dataset ---")
print(merged_data.info())


--- Merged Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price_x          1000 non-null   float64
 7   ProductName      1000 non-null   object 
 8   Category         1000 non-null   object 
 9   Price_y          1000 non-null   float64
 10  CustomerName     1000 non-null   object 
 11  Region           1000 non-null   object 
 12  SignupDate       1000 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB
None


In [10]:
# Top 10 Products by Quantity Sold
top_products = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(10)
fig1 = px.bar(
    top_products,
    title="Top 10 Products by Quantity Sold",
    labels={'value': 'Total Quantity', 'index': 'Product Name'},
    text=top_products.values
)
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.show()



In [14]:
#revenue distribution by product
product_revenue = merged_data.groupby('ProductName')['TotalValue'].sum().sort_values(ascending=False).head(10)
fig2 = px.pie(
    names=product_revenue.index,
    values=product_revenue.values,
    title="Revenue Distribution Among Top 10 Products",
    hole=0.4
)
fig2.update_traces(textinfo='percent+label')
fig2.show()


In [24]:
#analysing customer behaviour
region_distribution = merged_data['Region'].value_counts()
px.bar(region_distribution, title="Customers by Region", labels={'value': 'Number of Customers', 'index': 'Region'}).show()

merged_data['SignupDate'] = pd.to_datetime(merged_data['SignupDate'])
signup_trends = merged_data.groupby(merged_data['SignupDate'].dt.to_period('M'))['CustomerID'].nunique()
signup_trends.index = signup_trends.index.astype(str)
px.line(signup_trends, title="Monthly New Signups", labels={'index': 'Month', 'y': 'New Customers'}).show()

top_customers = merged_data.groupby('CustomerName')['TotalSpending'].mean().sort_values(ascending=False).head(10)
px.bar(top_customers, title="Top 10 High-Spending Customers", labels={'value': 'Total Spending', 'index': 'Customer Name'}).show()


In [26]:
#product insights
category_revenue = merged_data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
px.bar(category_revenue, title="Revenue by Category", labels={'value': 'Total Revenue', 'index': 'Category'}).show()


In [27]:
#transaction level analysis
merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'])
daily_revenue = merged_data.groupby(merged_data['TransactionDate'].dt.date)['TotalValue'].sum()
px.line(daily_revenue, title="Daily Revenue Trends", labels={'x': 'Date', 'y': 'Total Revenue ($)'}).show()

aov = merged_data['TotalValue'].mean()
print(f"Average Order Value (AOV): ${aov:.2f}")

Average Order Value (AOV): $690.00


In [29]:
#key metrics
total_revenue = merged_data['TotalValue'].sum()
print(f"Total Revenue Generated: ${total_revenue:,.2f}")

popular_products = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(5)
print("Most Popular Products by Quantity Sold:")
print(popular_products)

region_revenue = merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)
print("Regions Contributing Most Revenue:")
print(region_revenue.head(3))


Total Revenue Generated: $689,995.56
Most Popular Products by Quantity Sold:
ProductName
ActiveWear Smartwatch    100
SoundWave Headphones      97
HomeSense Desk Lamp       81
ActiveWear Rug            79
SoundWave Cookbook        78
Name: Quantity, dtype: int64
Regions Contributing Most Revenue:
Region
South America    219352.56
Europe           166254.63
North America    152313.40
Name: TotalValue, dtype: float64
